# HuggingFace with GPU
###### 참고
- [Hugging Face - Training on One GPU](https://huggingface.co/docs/transformers/perf_train_gpu_one)
- [Hugging Face - Inference on One GPU](https://huggingface.co/docs/transformers/perf_infer_gpu_one)

## Efficient Training on a Single GPU

더 적은 메모리 사용, 모델 학습 속도 상승, Trainer와 Accelerate가 통합되는 과정을 살펴본다.

아래의 각 method들은 속도 또는 메모리 사용 면에서 향상된다.

| Method | Speed | Memory | 비고 |
| --- | --- | --- |
| Gradient accumulation | No | Yes | 높은 batch size의 부하를 견디기 위해 모든 batch가 gradient를 Global Gradient에 누적시킨 뒤 한 번의 forward pass와 back propagation을 통해 전달 |
| Gradient checkpointing | No | Yes | 모델 학습 시 모든 노드의 가중치를 저장하지 않고 check point의 가중치만 저장하여 속도가 느린 대신 메모리 사용량이 줄어듦 |
| Mixed precision training | Yes | (No) | FP(Float Precision)16과 FP32를 혼용, FW / BW propagation은 FP16, 가중치를 업데이트하는 과정에서 다시 FP32로 변환해 메모리 사용을 줄이고 변환 과정의 오차로 인한 loss값도 줄임 |
| Batch size | Yes | Yes | - |
| Optimizer choice | Yes | Yes | - |
| DataLoader | Yes | No | - |
| Deepspeed Zero | No | Yes | 분산 학습 과정에서의 불필요한 메모리 중복을 제거하여 동시에 학습 가능한 파라미터의 수를 크레 늘릴 수 있는 새로운 병렬 최적화 도구 |

### Libraries
`pip install transformers datasets accelerate nvidia-ml-py3`

- accelerate: 같은 PyTorch 코드라도 4줄만 추가해 더 쉽고 효율적으로 사용할 수 있도록 만들어주는 라이브러리
- nvidia-ml-py3: 모델의 메모리 사용량을 파이썬에서 모니터링할 수 있는 라이브러리. 터미널의 nvidia-smi와 유사

In [1]:
# dummy data

import numpy as np
from datasets import Dataset

seq_len, dataset_size = 512, 512
dummy_data = {
    "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),
    "labels": np.random.randint(0, 1, dataset_size)
}
ds = Dataset.from_dict(dummy_data)
ds.set_format("pt")

In [13]:
# GPU 사용율 및 Trainer를 사용한 훈련 실행에 대한 요약 통계

from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

print_gpu_utilization()

GPU memory occupied: 695 MB.


In [14]:
# Load Model bert-large-uncased model

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased").to("cuda")
print_gpu_utilization()

C:\Users\jongg\PycharmProjects\HuggingFaceKoLLaMa13b\venv\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jongg\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU memory occupied: 2116 MB.


In [16]:
# default training args

default_args = {
    "output_dir": "tmp",
    "evaluation_strategy": "steps",
    "num_train_epochs": 1,
    "log_level": "error",
    "report_to": "none"
}

## Vanila Training

default_args, batch_size = 4, Trainer를 통해 모델 학습

In [17]:
from transformers import TrainingArguments, Trainer, logging

logging.set_verbosity_error() # error log activate

training_args = TrainingArguments(per_device_train_batch_size = 4, **default_args)
trainer = Trainer(model = model, args = training_args, train_dataset = ds)
result = trainer.train()
print_summary(result)

C:\Users\jongg\PycharmProjects\HuggingFaceKoLLaMa13b\venv\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'train_runtime': 111.7234, 'train_samples_per_second': 4.583, 'train_steps_per_second': 1.146, 'train_loss': 0.019563835114240646, 'epoch': 1.0}
Time: 111.72
Samples/second: 4.58
GPU memory occupied: 12183 MB.


batch_size가 4인데 GPU의 전체 메모리를 거의 가득 채우고 있다.

배치 크기가 클수록 모델 수렴 속도가 빨라지거나 최종 성능이 향상될 수 있다.

따라서 이상적으로는 GPU 제한이 아닌 모델의 요구사항에 맞게 배치 크기를 조정하고 싶으나 모델의 크기보다 훨씬 더 많은 메모리를 사용하고 있다.

그 이유를 더 잘 이해하기 위해 모델의 작동 및 메모리 요구사항을 살펴보자

# Model`s Operations

Transformers 아키텍처는 아래 3개 메인 그룹의 operation을 포함한다.

1. Tensor Contractions, 텐소 축소
    - Multi-Head Attention의 선형 레이어와 컴포넌트들은 모두 행렬X행렬의 배치 작업을 수행하는데 가장 compute-intensive한 부분이다.
2. Statistical Normalizations
    - Softmax와 레이어 정규화는 tensor contractions보다 덜 compute-intensive하지만 하나 이상의 축소 연산을 포함하며 그 결과는 맵을 통해 적용된다.
3. Element-wise Operators
    - biases, dropout, activations, residual connections는 그 외 가장 compute-intensive하지 않은 연산들이다.


# Model`s Memory

모델이 학습하는 동안 아래와 같은 수많은 컴포넌트가 사용되어 모델 크기에 비해 훨씬 많은 메모리가 사용된다.
1. 모델 가중치
2. optimizer states
3. gradient
4. forward activations saved for gradient computation
5. temporary buffers
6. functionality-specific memory

AdamW를 사용하여 혼합정밀도를 사용하는 기존 모델은 학습시마다 파라미터당 18바이트의 메모리가 필요하다. 추론을 위해 optimizer states와 gradient가 없으므로 이를 빼면 혼합 정밀도 추론을 위한 모델 파라미터당 6바이트와 활성화 메모리로 끝난다.
1. Model Weights
    - FP32 훈련시 파라미터당 4바이트
    - FP16 & FP32 혼합 정밀도 훈련시 파라미터당 6바이트
2. Optimizer States
    - 기존 AdamW와 같이 2state 유지하는 옵티마이저 사용시 파라미터당 8바이트
    - bitsandbytes와 같은 8-bit 옵티마이저 사용시 파라미터당 2바이트
    - momentum을 사용하는 SGD 옵티마이저와 같이 1state만 유지하는 경우 파라미터당 4바이트
3. Gradients:
4. Forward Activations:
5. Temporary Memory:
6. Fuctionality-specific memory

## BetterTransformer: PyTorch-native transformer fastpath

Inference 전용 Transformer.

Transformer의 encoder, encoder layer, multi head attention을 다음 대표적인 두 방법으로 가속화.
1. fused kernel: 효율 up
2. exploiting sparsity in the inputs: pad_token과 같이 불필요한 부분을 생략

\* PyTorch 1.13 이상의 버전에서 호환
`pip install accelerate optimum`


In [10]:
# Garbage Collect

import gc
gc.collect()

1331